## **Machine Learning - WNBA Playoffs Prediction**
This notebook will focus on the undestanding of the data. We will be using SQLite to store the data due to its scalability & the fact that it's a relational schema.

https://docs.python.org/3/library/sqlite3.html

Import sqlite3 and connect to database file

### **Imports**

In [1]:
import pandas as pd
import sqlite3
import prep_utils as pu 
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt

### **Database Connection Setup**

In [2]:
db = sqlite3.connect("db/ac.db")
db_cur = db.cursor()

[df_awards, df_coaches, df_players_teams, df_players, df_series_post, df_teams_post, df_teams] = pu.db_to_pandas(db)

In [3]:
df_new_coaches = pu.prepare_coaches(df_coaches, df_awards,1)
#print(df_new_coaches.to_string())
#df_new_players = pu.prepare_players(df_players,df_players_teams)

pu.group_coaches(df_new_coaches)



Dropping Attribute lgID in Coaches...
Creating attribute coach previous regular season win ratio...
Creating attribute coach playoffs win ratio...
Creating attribute coach playoffs count...
Creating attribute coach awards count...
Dropping attribute post_wins..
Dropping attribute post_losses..
Dropping attribute won..
Dropping attribute lost..

Coaches Null Verification:
year                    0
tmID                    0
coachID                 0
coach_reg_season_wr     0
coach_po_season_wr      0
coach_playoffs_count    0
coach_awards            0
dtype: int64


,year,tmID,coachID,coach_reg_season_wr,coach_po_season_wr,coach_playoffs_count,coach_awards
0,1,CHA,dunntr01wc,0.000000,0.000000,0,0
1,1,CLE,hugheda99w,0.000000,0.000000,0,0
2,1,DET,liebena01w,0.000000,0.000000,0,0
3,1,HOU,chancva99w,0.000000,0.000000,0,0
4,1,IND,donovan99w,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...
137,10,PHO,gaineco01w,0.470588,0.000000,0,0
138,10,SAC,bouceje01w,0.529412,0.333333,2,0
139,10,SAS,hugheda99w,0.705882,0.444444,5,2
140,10,SEA,aglerbr99w,0.647059,0.333333,1,0


In [4]:
pu.prepare_teams(df_teams)

Dropping divID in Teams...
Dropping ldID in Teams...
Dropping seeded in Teams...
Dropping tmORB, tmDRB, tmTRB, opptmORB, opptmDRB, opptmTRB in Teams...


In [5]:
pu.best_colleges(df_players_teams,df_teams,df_players)

,college,TotalPlayoffAppearances,CollegeRank
88,Tennessee,21,1
17,Connecticut,17,2
31,Georgia,15,3
86,Stanford,12,4
48,Louisiana Tech,11,5
...,...,...,...
80,Seton Hall,1,14
25,Florida International,1,14
82,Southern Mississippi,1,14
50,Maine,1,14


### **Data Preparation**
We will preparate the data in each table, by cleaning & formatting it so that it can be easily used by the machine learning models afterwards.